In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("diabetes.csv")
X = df.drop("Outcome", axis=1).values
y = df["Outcome"].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
def build_model():
    model = Sequential([
        Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model = build_model()
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5172 - loss: 0.7300 - val_accuracy: 0.4675 - val_loss: 0.7166
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6187 - loss: 0.6828 - val_accuracy: 0.5325 - val_loss: 0.6897
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6477 - loss: 0.6550 - val_accuracy: 0.6104 - val_loss: 0.6664
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6646 - loss: 0.6327 - val_accuracy: 0.6429 - val_loss: 0.6455
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6834 - loss: 0.6198 - val_accuracy: 0.7013 - val_loss: 0.6269
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7265 - loss: 0.5879 - val_accuracy: 0.7078 - val_loss: 0.6119
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7416 - loss: 0.5750 - val_accuracy: 0.7143 - val_loss: 0.5951
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7406 - loss: 0.5533 - val_accuracy: 0.7403 - val_loss

In [9]:
from keras_tuner import RandomSearch

In [10]:
def build_tuned_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=8, max_value=64, step=8), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(hp.Int('units2', min_value=4, max_value=32, step=4), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [11]:
# Define tuner
tuner = RandomSearch(
    build_tuned_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='tuning_dir',
    project_name='diabetes_tuning'
)


In [12]:
# Run tuning
tuner.search(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 01m 02s


In [15]:
# Run tuning
tuner.search(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: units={best_hps.get('units')}, units2={best_hps.get('units2')}, learning_rate={best_hps.get('learning_rate')}")

Best hyperparameters: units=24, units2=32, learning_rate=0.001


In [13]:
# ================== Hyperparameter Tuning (Advanced - Bayesian Optimization) ==================
from keras_tuner import BayesianOptimization

tuner_bo = BayesianOptimization(
    build_tuned_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuning_dir',
    project_name='diabetes_bayesian_tuning'
)

tuner_bo.search(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 17s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7889610230922699
Total elapsed time: 00h 02m 41s


In [16]:
# Get the best hyperparameters for Bayesian Optimization
best_hps_bo = tuner_bo.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters (Bayesian Optimization): units={best_hps_bo.get('units')}, units2={best_hps_bo.get('units2')}, learning_rate={best_hps_bo.get('learning_rate')}")


Best hyperparameters (Bayesian Optimization): units=56, units2=8, learning_rate=0.01


In [14]:
# ================== Hyperparameter Tuning (Complex - Hyperband) ==================
from keras_tuner import Hyperband

tuner_hb = Hyperband(
    build_tuned_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='tuning_dir',
    project_name='diabetes_hyperband_tuning'
)

tuner_hb.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Trial 90 Complete [00h 00m 17s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 08m 58s


In [17]:

# Get the best hyperparameters for Hyperband
best_hps_hb = tuner_hb.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters (Hyperband): units={best_hps_hb.get('units')}, units2={best_hps_hb.get('units2')}, learning_rate={best_hps_hb.get('learning_rate')}")

Best hyperparameters (Hyperband): units=16, units2=12, learning_rate=0.01
